In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances

In [2]:
facilities_df = pd.read_csv('healthcare_facilities.csv')
hurricane_df = pd.read_csv('HURDAT2_hurricane_data_with_events.csv')  

In [3]:
print(facilities_df.head())
print(hurricane_df.head())

                       name          type   latitude  longitude
0              CVS Pharmacy      pharmacy  27.948538 -82.458507
1  Miami Lakes Fire Station  fire_station  25.924045 -80.308954
2                  7-Eleven          fuel  27.981761 -82.452878
3              CVS Pharmacy      pharmacy  28.546148 -81.204455
4                 Walgreens      pharmacy  28.547299 -81.204692
   storm_id storm_name      date  time record_id event_label status  latitude  \
0  AL011851    UNNAMED  18510625     0       NaN         NaN     HU      28.0   
1  AL011851    UNNAMED  18510625   600       NaN         NaN     HU      28.0   
2  AL011851    UNNAMED  18510625  1200       NaN         NaN     HU      28.0   
3  AL011851    UNNAMED  18510625  1800       NaN         NaN     HU      28.1   
4  AL011851    UNNAMED  18510625  2100         L    Landfall     HU      28.2   

   longitude  max_wind  min_pressure  
0      -94.8        80          -999  
1      -95.4        80          -999  
2      -96.0

In [4]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [6]:
import numpy as np

# Extract coordinates
facility_coords = facilities_df[['latitude', 'longitude']].values
hurricane_coords = hurricane_df[['latitude', 'longitude']].values

# Define batch size for progress tracking
batch_size = 100  # Adjust batch size as needed
num_batches = len(facility_coords) // batch_size + 1  # Total number of batches

# Initialize array to store minimum distances
min_distances = []

# Process facilities in chunks
for i in range(0, len(facility_coords), batch_size):
    # Define the current batch
    batch_end = min(i + batch_size, len(facility_coords))
    current_batch = facility_coords[i:batch_end]

    # Compute pairwise distances for the current batch
    distances = pairwise_distances(current_batch, hurricane_coords, metric=lambda u, v: haversine(u[0], u[1], v[0], v[1]))

    # Compute the minimum distance for each facility in the batch
    batch_min_distances = distances.min(axis=1)
    min_distances.extend(batch_min_distances)

    # Print progress
    print(f"Processed facilities {i + 1}-{batch_end} out of {len(facility_coords)}")

# Convert to a NumPy array
min_distances = np.array(min_distances)

# Add distances and risk scores to the dataframe
facilities_df['min_distance'] = min_distances
facilities_df['risk_score'] = 1 / (min_distances + 1)  # Add 1 to avoid division by zero


Processed facilities 1-100 out of 3946
Processed facilities 101-200 out of 3946
Processed facilities 201-300 out of 3946
Processed facilities 301-400 out of 3946
Processed facilities 401-500 out of 3946
Processed facilities 501-600 out of 3946
Processed facilities 601-700 out of 3946
Processed facilities 701-800 out of 3946
Processed facilities 801-900 out of 3946
Processed facilities 901-1000 out of 3946
Processed facilities 1001-1100 out of 3946
Processed facilities 1101-1200 out of 3946
Processed facilities 1201-1300 out of 3946
Processed facilities 1301-1400 out of 3946
Processed facilities 1401-1500 out of 3946
Processed facilities 1501-1600 out of 3946
Processed facilities 1601-1700 out of 3946
Processed facilities 1701-1800 out of 3946
Processed facilities 1801-1900 out of 3946
Processed facilities 1901-2000 out of 3946
Processed facilities 2001-2100 out of 3946
Processed facilities 2101-2200 out of 3946
Processed facilities 2201-2300 out of 3946
Processed facilities 2301-2400 o

In [7]:
facilities_df.to_csv('facilities_with_risk_scores.csv', index=False)


In [8]:
facilities_df.head()

,name,type,latitude,longitude,min_distance,risk_score
0,CVS Pharmacy,pharmacy,27.948538,-82.458507,7.024852,0.124613
1,Miami Lakes Fire Station,fire_station,25.924045,-80.308954,2.819645,0.261804
2,7-Eleven,fuel,27.981761,-82.452878,5.051796,0.165240
3,CVS Pharmacy,pharmacy,28.546148,-81.204455,10.652014,0.085822
4,Walgreens,pharmacy,28.547299,-81.204692,10.694056,0.085514
